In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.reset_default_graph()

In [0]:
char_arr = [c for c in 'helo']
num_dic = {n:i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

input_batch = []
target_batch =[]

In [0]:
char_arr

['h', 'e', 'l', 'o']

In [0]:
dic_len

4

In [0]:
num_dic

{'e': 1, 'h': 0, 'l': 2, 'o': 3}

In [0]:
input1 = [num_dic[n] for n in 'helloh']
target1 = [num_dic[n] for n in 'elloho'] # 트레이닝 시
#target2 = [num_dic[n] for n in 'PPPP'] # 테스트 시
#input_batch.append(np.eye(dic_len)[input1])
#target_batch.append(np.eye(dic_len)[target1])

In [0]:
target1

array([[1, 2, 2, 3, 0, 3]])

In [0]:
input1

[0, 1, 2, 2, 3, 0]

In [0]:
input_batch = np.eye(dic_len)[input1]
target_batch = np.eye(dic_len)[target1]

In [0]:
np.eye(dic_len)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [0]:
input_batch

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

In [0]:
target_batch

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [0]:
learning_rate = 0.01
n_hidden = 12
#total_epoch = 100
n_step = 6 #최대 4개의 input만 받는
# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
n_class = dic_len
n_input = dic_len

In [0]:
input = tf.placeholder(tf.float32, [1,n_step, n_input], name='input')
target = tf.placeholder(tf.float32, [1, n_step], name='target') # target은 one hot encoding이 아님

In [0]:
target.shape

TensorShape([Dimension(1), Dimension(6)])

In [0]:
input.shape

TensorShape([Dimension(1), Dimension(6), Dimension(4)])

In [0]:
# RNN 셀을 구성한다.
with tf.variable_scope('rnn_cell'):
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    #rnn_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(rnn_cell, input,
                                            dtype=tf.float32)

In [0]:
outputs.shape

TensorShape([Dimension(1), Dimension(6), Dimension(12)])

In [0]:
enc_states.shape

TensorShape([Dimension(1), Dimension(12)])

In [0]:
model1 = tf.layers.dense(outputs, n_class, activation=None)
model = tf.reshape(model1, [n_step,n_class])
#prediction = tf.argmax(model, 0)
#m = tf.identity(model, name='m')
#p = tf.identity(prediction, name='p')

cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=target1))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
model1.shape

TensorShape([Dimension(1), Dimension(6), Dimension(4)])

In [0]:
model.shape

TensorShape([Dimension(6), Dimension(4)])

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch = input_batch.reshape([1,n_step, n_input])
target1 = np.reshape(target1, [1,n_step])

for i in range(10000):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={input: input_batch,
                                  target: target1})
    if i%100 == 0:
      print(i, 'cost', '{:.6f}'.format(loss))

0 cost 1.667477
100 cost 0.011797
200 cost 0.003654
300 cost 0.001840
400 cost 0.001126
500 cost 0.000768
600 cost 0.000560
700 cost 0.000429
800 cost 0.000340
900 cost 0.000276
1000 cost 0.000229
1100 cost 0.000193
1200 cost 0.000165
1300 cost 0.000142
1400 cost 0.000124
1500 cost 0.000109
1600 cost 0.000097
1700 cost 0.000086
1800 cost 0.000077
1900 cost 0.000069
2000 cost 0.000063
2100 cost 0.000057
2200 cost 0.000052
2300 cost 0.000047
2400 cost 0.000043
2500 cost 0.000040
2600 cost 0.000037
2700 cost 0.000034
2800 cost 0.000031
2900 cost 0.000029
3000 cost 0.000027
3100 cost 0.000025
3200 cost 0.000023
3300 cost 0.000021
3400 cost 0.000020
3500 cost 0.000019
3600 cost 0.000017
3700 cost 0.000016
3800 cost 0.000015
3900 cost 0.000014
4000 cost 0.000013
4100 cost 0.000013
4200 cost 0.000012
4300 cost 0.000011
4400 cost 0.000010
4500 cost 0.000010
4600 cost 0.000009
4700 cost 0.000009
4800 cost 0.000008
4900 cost 0.000008
5000 cost 0.000007
5100 cost 0.000007
5200 cost 0.000006
5300 

In [0]:
prediction = tf.argmax(model, 1)
target2 = np.reshape(target2, [1,n_step])
result = sess.run([prediction],
                       feed_dict={input: input_batch, target: target2})
decoded = [char_arr[i] for i in result[0]]

print(decoded)

ValueError: ignored